In [54]:
from pyomo.environ import *

#### 1借助表达式定义单条约束

In [55]:
model = ConcreteModel()
model.I  = Set(initialize = [i for i in range(3)])
model.J  = Set(initialize = [i for i in range(3)])
model.x = Var(model.I, model.J, initialize = 1.0, within = Reals)

model.c1 = Constraint(expr = 2.0 <= model.x[0,0] + 2*model.x[0,2]) # 不等式约束 2 <= x[0,0] + 2*x[0,2] 
model.c2 = Constraint(expr = 3*model.x[0,0] + 5*model.x[1,1] == 10.0)  # 等式约束 3*x[0,0] + 5*x[1,1] = 10.0
model.pprint()

3 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    J : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :    9 : {(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)}

1 Var Declarations
    x : Size=9, Index=x_index
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (0, 0) :  None :   1.0 :  None : False : False :  Reals
        (0, 1) :  None :   1.0 :  None : False : False :  Reals
        (0, 2) :  None :   1.0 :  None : False : False :  Reals
        (1, 0) :  None :   1.0 :  None : False : False :  Reals
        (1, 1) :  None :   1.0 :  None : False : False :  Reals
        (1, 2) :  None :   1.0 :  None : False : False : 

In [56]:
print(value(model.c1.body)) # 3 返回c1表达式的值 
print(model.c1.lslack()) # 1.0 返回c1约束和下限的差(lower bound sla1ck variable)
print(model.c1.uslack()) #  +inf 返回c1约束和上限的差(upper bound slack variable)
print(model.c2.lslack()) # -2.0 返回c2约束和下限的差(lower bound slack variable)
print(model.c2.uslack()) #  2.0 返回c1约束和下限的差(upper bound slack variable)

3.0
1.0
inf
-2.0
2.0


#### 2借用函数定义单条约束

In [57]:
model.p = Param(model.I, model.J, default = 2) 
def c3_rule(model):
    return summation(model.p, model.x) + model.x[0,0]**2 <= 2

model.c3 = Constraint(rule = c3_rule)
model.pprint()

4 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    J : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    p_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :    9 : {(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :    9 : {(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)}

1 Param Declarations
    p : Size=9, Index=p_index, Domain=Any, Default=2, Mutable=False
        Key : Value

1 Var Declarations
    x : Size=9, Index=x_index
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (0, 0) :  None :   1.0 :  None : False : False :  Rea

#### 3借用函数定义多条约束

In [58]:
def c4_rule(model, i):
    if i == 0:
        return Constraint.Skip
    return sum([model.p[i,j]*model.x[i,j] for j in model.J]) <= 2

model.c4 = Constraint(model.I, rule = c4_rule)
model.pprint()

4 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    J : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    p_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :    9 : {(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :    9 : {(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)}

1 Param Declarations
    p : Size=9, Index=p_index, Domain=Any, Default=2, Mutable=False
        Key : Value

1 Var Declarations
    x : Size=9, Index=x_index
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (0, 0) :  None :   1.0 :  None : False : False :  Rea

In [59]:
model.c2.deactivate() # c2约束失效
model.pprint()

4 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    J : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    p_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :    9 : {(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :    9 : {(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)}

1 Param Declarations
    p : Size=9, Index=p_index, Domain=Any, Default=2, Mutable=False
        Key : Value

1 Var Declarations
    x : Size=9, Index=x_index
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (0, 0) :  None :   1.0 :  None : False : False :  Rea

In [60]:
model.c2.activate() #重新激活c2约束
model.pprint()

4 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    J : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    p_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :    9 : {(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :    9 : {(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)}

1 Param Declarations
    p : Size=9, Index=p_index, Domain=Any, Default=2, Mutable=False
        Key : Value

1 Var Declarations
    x : Size=9, Index=x_index
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (0, 0) :  None :   1.0 :  None : False : False :  Rea